In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/08/30 19:37:47 WARN Utils: Your hostname, Teddys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.13 instead (on interface en0)
23/08/30 19:37:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/30 19:37:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price_year = spark.sql("SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY year")
avg_price_year.show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
build_price = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built")
build_price.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016|290555.07|
|      2012|293683.19|
|      2017|292676.79|
|      2014|290852.27|
|      2013|295962.27|
|      2011|291117.47|
|      2015| 288770.3|
|      2010|292859.62|
+----------+---------+



In [9]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
dream_houses = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built")
dream_houses.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016| 293965.1|
|      2012|307539.97|
|      2017|280317.58|
|      2014|298264.72|
|      2013|303676.79|
|      2011|276553.81|
|      2015|297609.97|
|      2010|285010.22|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = spark.sqlresult = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
view_rating.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.7580268383026123 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view_rating_cached = spark.sqlresult = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
view_rating_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.39962196350097656 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sale")

In [15]:
# 11. Read the formatted parquet data.
home_sales=spark.read.parquet("home_sale")

In [16]:
# 12. Create a temporary table for the parquet data.
home_sales.createOrReplaceTempView("home_sales_parquet")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

view_rating_cached = spark.sqlresult = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
view_rating_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.3022572994232178 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False